In [135]:
import pandas as pd

f = pd.read_csv('br_anatel_telefonia_movel_ddd.csv')

class Utils:
    def format_large_numbers_in_a_list(l:list) -> list:
        for i in range(0, len(l)):
            n = l[i]

            t = 1_000_000_000_000
            b = 1_000_000_000
            m = 1_000_000
            mil = 1_000

            if n >= t:
                l[i] = f'{n/t:.3f} Tri'
            elif n >= b:
                l[i] = f'{n/b:.3f} Bi'
            elif n >= m:
                l[i] = f'{n/m:.3f} M'
            elif n >= mil:
                l[i] = f'{n/mil:.3f} Mil'
            else:
                l[i] = str(l[i])
        
        return l
    
    def format_large_number(n: float) -> str:
        t = 1_000_000_000_000
        b = 1_000_000_000
        m = 1_000_000
        mil = 1_000

        if n >= t:
            n = f'{n/t:.3f} Tri'
        elif n >= b:
            n = f'{n/b:.3f} Bi'
        elif n >= m:
            n = f'{n/m:.3f} M'
        elif n >= mil:
            n = f'{n/mil:.3f} Mil'
        else:
            n = str(n)
        
        return n
    
    def plotBarGraphic(title:str, x_title:str, y_title:str, x:list, y:list, ytext:list, x_angle = 0) -> None:
        import plotly.graph_objs as go
        import plotly.io as pio

        bg = go.Figure(data=[go.Bar(x=x, y=y, text=ytext, textposition='auto')])

        bg.update_layout(
            title=title,
            title_x=0.5,
            xaxis_title=x_title,
            yaxis_title=y_title,
            xaxis_tickangle=x_angle,
        )

        pio.show(bg)

    def plotPieGraphic(title:str, labels:list, values:list, texts:list=None, textinfo='label+text', textposition='outside', hoverinfo='label+percent', forceTextsInside=False) -> None:
        import plotly.graph_objs as go
        import plotly.io as pio

        pg = go.Figure(data=[go.Pie(labels=labels, values=values, text=texts, textinfo=textinfo, hoverinfo=hoverinfo, textposition=textposition)])
        if texts is not None and forceTextsInside is True:
            # pg.add_trace(go.Pie(labels=[''] * len(labels), values=values, text=texts, textinfo='text', textposition='inside', hoverinfo='label+text+percent'))
            pg.add_trace(go.Pie(labels=labels, values=values, text=texts, textinfo='text', textposition='inside', hoverinfo='label+text+percent'))
        pg.update_layout(
            title_text=title,
            title_x = 0.5
        )

        pio.show(pg)

class Graphic:
    def tech_signal_all_time_access():
        import numpy as np
        f['tecnologia'].str.strip()
        f['sinal'].str.strip()

        tsg = f.groupby(['tecnologia', 'sinal'])

        x_y = {}

        for (t,s), g in tsg:
            x_y[f'{t} - {s}'] = g['acessos'].sum()
        
        x_y_copy = {}
        for v in sorted(list(x_y.values())):
            for k, val in x_y.items():
                if v == val:
                    x_y_copy[k] = v
        
        x_y = x_y_copy
        
        x = list(x_y.keys())
        y = list(x_y.values())

        yf = Utils.format_large_numbers_in_a_list(y.copy())

        e = f['ano'].max()
        s = f['ano'].min()
        i = e - s

        Utils.plotBarGraphic(
            title = f"Total de acessos por tipo TECNOLOGIA & SINAL nos últimos {i} anos ({s} - {e})",
            x_title = 'Tecnologia - Sinal',
            y_title = 'T. Acessos',
            x = x,
            y = y,
            ytext = yf,
            x_angle = -45
        )

        x_y_copy = dict(sorted(x_y_copy.items(), key=lambda e: e[1], reverse=True))

        pi = {}
        o = {}

        for k,v in x_y_copy.items():
            p = (v/(f['acessos'].sum()))*100
            if p <= 2:
                o[k] = v
            else:
                pi[k] = v
        
        t = 0

        for k,v in o.items():
            t += v

        title = f'{len(o)} TECNOLOGIAS & SINAIS distintas'
        
        pi[title] = t

        x = list(pi.keys())
        y = list(pi.values())
        d = []

        for v in pi.values():
            p = (v / f['acessos'].sum()) * 100
            d.append(f'{Utils.format_large_number(v)} ({p:.2f}%)')

        Utils.plotPieGraphic(
            title = f'Distribuição de acessos por TECNOLOGIA & SINAL nos últimos {i} anos ({s} - {e})',
            labels = x,
            values = y,
            texts = d
        )

        x = list(o.keys())
        y = list(o.values())
        d = []

        for v in o.values():
            p = (v / f['acessos'].sum()) * 100
            d.append(f'{Utils.format_large_number(v)} ({p:.4f}%)')
        
        Utils.plotPieGraphic(
            title=f'As 11 TECNOLOGIAS E SINAIS DISTINTOS: Os "{(sum(o.values()) / f['acessos'].sum())*100:.2f}%"',
            labels=x,
            values=y,
            texts=d,
            textinfo='label+text',
            hoverinfo='percent'
        )
        
    
    def signal_all_time_access():
        f['sinal'].str.strip()

        sg = f.groupby('sinal')

        x_y = {}

        for (s), g in sg:
            x_y[s] = g['acessos'].sum()
        
        x_y = dict(sorted(x_y.items(), key= lambda e: e[1]))

        x = list(x_y.keys())
        y = list(x_y.values())

        yf = Utils.format_large_numbers_in_a_list(y.copy())
    
        e = f['ano'].max()
        s = f['ano'].min()
        i = e - s

        Utils.plotBarGraphic(
            title = f"Total de acessos por tipo de SINAL nos últimos {i} anos ({s} - {e})",
            x_title = 'Sinal',
            y_title = 'T. Acessos',
            x = x,
            y = y,
            ytext = yf,
        )

        Utils.plotPieGraphic(
            title = f'Distribuição de acessos por tipo de SINAL nos últimos {i} anos ({s} - {e})',
            labels = x,
            values = y,
            texts = [Utils.format_large_number(v) for v in y],
            textinfo='label+percent',
            textposition='outside',
            forceTextsInside=True
        )
    
    def all_time_access_per_region():
        r = {
            "nordeste": {'MA', 'PI', 'CE', 'RN', 'PB', 'PE', 'AL', 'SE', 'BA'},
            "sudeste": {'SP', 'RJ', 'ES', 'MG'},
            "centro_oeste": {'DF', 'GO', 'MT', 'MS'},
            "norte" : {'AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'},
            "sul" : {'PR', 'RS', 'SC'}
        }

        t = 0

        for k, v in r.items():
            ra = f[f['sigla_uf'].isin(v)].groupby('sigla_uf')['acessos'].sum().sum()
            t += ra
            r[k] = ra
        
        if t == f['acessos'].sum():
            r_copy = dict(sorted(r.items(), key=lambda e: e[1]))
            r = r_copy

            x = list(r.keys())
            y = list(r.values())

            yf = Utils.format_large_numbers_in_a_list(y.copy())

            e = f['ano'].max()
            s = f['ano'].min()
            i = e - s

            Utils.plotBarGraphic(
                title = f"Total de acessos por REGIÃO nos últimos {i} anos ({s} - {e})",
                x_title = 'Região',
                y_title = 'T. Acessos',
                x = x,
                y = y,
                ytext = yf
            )

            Utils.plotPieGraphic(
                title = f'Distribuição de acessos por REGIÃO nos últimos {i} anos ({s} - {e})',
                labels = x,
                values = y,
                texts=[f'{(v/f['acessos'].sum())*100:.2f}%' for v in y],
                textinfo='label',
                forceTextsInside=True
            )






Graphic.tech_signal_all_time_access()
Graphic.signal_all_time_access()
Graphic.all_time_access_per_region()
